In [1]:
# -*- coding: utf-8 -*-

import os
os.chdir('/home/zl/develop/projects/hbc')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, ensemble, metrics, grid_search, model_selection, decomposition, linear_model, neighbors
from matplotlib.pyplot import rcParams
from sklearn.datasets import load_svmlight_file
import lightgbm as lgb
import copy
plt.rcParams['font.sans-serif']=['SimHei']

/home/zl/dev/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/zl/dev/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def modelfit(alg, X_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    #print('\n-------------------------------------')
    # fit the algorithm on the data
    alg.fit(X_train, y_train, eval_metric='auc')
    # predict train set
    #dtrain_pred = alg.predict(X_train)
    #dtrain_predprob = alg.predict_proba(X_train)[:,1]
    # print model report
    #print('score_auc:', round(metrics.roc_auc_score(y_train, dtrain_predprob), 5))
    #print('score_precision:', round(metrics.accuracy_score(y_train, dtrain_pred), 5))
    if useTrainCV:
        np.random.seed(314)
        scores_cv_auc = model_selection.cross_val_score(alg, X_train, y_train, cv=5, scoring='roc_auc')
        #print('score_cv_auc:', round(np.mean(scores_cv_auc), 5), 'std:', round(np.std(scores_cv_auc), 5))
    #print('-------------------------------------\n')
    return np.mean(scores_cv_auc)

In [3]:
# load training&test set
df_train = pd.read_csv('./result/data_train.csv')
df_test = pd.read_csv('./result/data_test.csv')

idcol = 'userid'
target = 'futureOrderType'
drop_tags = [idcol, target]
df_train = df_train.fillna(-999)
df_test = df_test.fillna(-999)

x_tags = [x for x in df_train.columns if x not in drop_tags]
y_tag = target

X_train = np.array(df_train[x_tags])
y_train = np.array(df_train[y_tag])
X_test = np.array(df_test[x_tags])

In [4]:
df_train.head()

,userid,futureOrderType,histord_sum_all,histord_sum_0,histord_sum_1,histord_ratio1_0,histord_time_last_1,histord_time_last_1_year,histord_time_last_1_month,histord_time_last_1_hour,...,timespan2last_action6_indicator1,timespan2last_action7_indicator1,timespan2last_action8_indicator1,timespan2last_action9_indicator1,timespan2last_action1_indicator2,timespan2last_action5_indicator2,timespan2last_action6_indicator2,timespan2last_action7_indicator2,timespan2last_action8_indicator2,timespan2last_action9_indicator2
0,100000000013,0,2.0,2.0,0.0,0.333333,1.501960e+09,2017.0,8.0,3.0,...,0.000000e+00,5.195326e+09,5.508683e+09,5.853648e+09,0.000000e+00,1.316626e+11,0.000000e+00,1.194925e+11,1.211910e+11,1.229266e+11
1,100000000111,0,0.0,0.0,0.0,1.000000,-9.990000e+02,-999.0,-999.0,-999.0,...,0.000000e+00,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,0.000000e+00,0.000000e+00,-9.990000e+02,-9.990000e+02,-9.990000e+02
2,100000000127,0,0.0,0.0,0.0,1.000000,-9.990000e+02,-999.0,-999.0,-999.0,...,-9.990000e+02,0.000000e+00,-9.990000e+02,-9.990000e+02,1.174871e+13,1.758036e+12,-9.990000e+02,0.000000e+00,-9.990000e+02,-9.990000e+02
3,100000000231,0,0.0,0.0,0.0,1.000000,-9.990000e+02,-999.0,-999.0,-999.0,...,1.507874e+07,-9.990000e+02,-9.990000e+02,-9.990000e+02,1.500000e+03,4.162842e+07,4.523623e+07,-9.990000e+02,-9.990000e+02,-9.990000e+02
4,100000000379,0,0.0,0.0,0.0,1.000000,-9.990000e+02,-999.0,-999.0,-999.0,...,0.000000e+00,1.426621e+11,-9.990000e+02,-9.990000e+02,0.000000e+00,2.947689e+10,0.000000e+00,5.421159e+12,-9.990000e+02,-9.990000e+02


In [5]:
model_gbm = lgb.LGBMClassifier(
        random_state=314,
        learning_rate=0.1,
        n_estimators=300,
        colsample_bytree=0.8,
        subsample=0.9
        )
x_tags = pd.read_csv('./feature/x_tags4.csv')['x_tag'].tolist()
X_train = np.array(df_train[x_tags])
y_train = np.array(df_train[y_tag])
X_test = np.array(df_test[x_tags])
#print(modelfit(model_gbm, X_train, y_train))

In [ ]:
# forwoard search best feature set
feature_name_list = pd.read_csv('./feature/x_tags4.csv')['x_tag'].tolist()
#feature_name_list = [x for x in df_train.columns if x not in drop_tags]
df_x_tags = pd.read_csv('./feature/forward_search/feat_round_21.csv')
x_tags_best = list(df_x_tags['x_tag'])
#x_tags_best = []
score_cv_auc_best_prev = 0.0
score_cv_auc_best = 0.0
x_tags_tmp = []
for k in range(len(x_tags_best), len(feature_name_list)):
    print('current round:', (k+1))
    score_cv_auc_best_prev = score_cv_auc_best
    x_tags_tmp = copy.deepcopy(x_tags_best)
    for i in range(0, len(feature_name_list)):
        if feature_name_list[i] in x_tags_best:
            continue
        x_tags_tmp.append(feature_name_list[i])
        X_train = df_train[x_tags_tmp]
        
        score_cv_auc_tmp = modelfit(model_gbm, X_train, y_train)
        
        print('feature_id:', i, '\t', feature_name_list[i], '\t', 'score_cv_auc:', round(score_cv_auc_tmp, 5))
        if score_cv_auc_tmp > score_cv_auc_best:
            x_tags_best = copy.deepcopy(x_tags_tmp)
            score_cv_auc_best = score_cv_auc_tmp
        x_tags_tmp.pop()
    if score_cv_auc_best < score_cv_auc_best_prev:
        print('Do not need more features')
        break
    else:
        print('\n----------------------------------------')
        print(x_tags_best)
        print('\n--- score_auc_current:', score_cv_auc_best, '---\n')
        df_x_tags_best = pd.DataFrame({'x_tag':x_tags_best})
        df_x_tags_best.to_csv('./feature/forward_search/feat_round_'+str(k+1)+'.csv', index=False)
        print('----------------------------------------\n')

current round: 22
feature_id: 2 	 histord_ratio1_0 	 score_cv_auc: 0.96024
feature_id: 3 	 histord_time_last_1_month 	 score_cv_auc: 0.9601
feature_id: 4 	 histord_time_last_1_hour 	 score_cv_auc: 0.95994
feature_id: 6 	 histord_sum_cont1 	 score_cv_auc: 0.95964
feature_id: 7 	 histord_sum_cont2 	 score_cv_auc: 0.96031
feature_id: 8 	 histord_sum_cont3 	 score_cv_auc: 0.96012
feature_id: 9 	 histord_sum_cont4 	 score_cv_auc: 0.95991
feature_id: 10 	 action_sum 	 score_cv_auc: 0.96055
feature_id: 12 	 actiontype_last_2 	 score_cv_auc: 0.95992
feature_id: 13 	 actiontype_last_3 	 score_cv_auc: 0.96041
feature_id: 15 	 actiontype_last_5 	 score_cv_auc: 0.96023
feature_id: 16 	 actiontype_last_6 	 score_cv_auc: 0.95961
feature_id: 17 	 actiontype_last_7 	 score_cv_auc: 0.96002
feature_id: 18 	 actiontype_last_8 	 score_cv_auc: 0.9599
feature_id: 19 	 actiontype_last_9 	 score_cv_auc: 0.96022
feature_id: 20 	 actiontype_last_10 	 score_cv_auc: 0.95982
feature_id: 21 	 actiontimespancount_1_

In [ ]:
df_x_tags = pd.read_csv('./feature/forward_search/feat_round_16.csv')
x_tags_best = list(df_x_tags['x_tag'])
len(x_tags_best)